In [106]:
%matplotlib inline
# from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import ast
import cv2
import skimage

from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
#models
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

#evaluation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
# #arr_0 is labels, arr_1 is images, arr_2 is hog features
# with np.load('data.npz') as data:
#     labels = data['arr_0']
#     imgs = data['arr_1']
#     hog_feats = data['arr_2']

In [113]:
#arr_0 is labels, arr_1 is images, arr_2 is hog features
with np.load('data (1).npz') as data:
    labels = data['arr_0']
    imgs = data['arr_1']
    hog_feats = data['arr_2']

In [114]:
labels.shape

(2966,)

In [115]:
imgs.shape

(2966, 128, 128)

In [116]:
hog_feats.shape

(2966, 128, 128)

In [117]:
hog_feats = hog_feats.reshape(hog_feats.shape[0],hog_feats.shape[1]*hog_feats.shape[2])
hog_feats.shape

(2966, 16384)

In [6]:
# def extract_hog_features(data):
#     num_samples = data.shape[0]
#     hog_features = []
#     for i in range(num_samples):
#         img = data[i]
#         feature = hog(img, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(3, 3))
#         hog_features.append(feature)
#     return np.array(hog_features)

In [118]:
images = imgs.reshape(imgs.shape[0],imgs.shape[1]*imgs.shape[2])
images.shape

(2966, 16384)

In [10]:
# #split train and test
# x_train,x_test,y_train,y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [16]:
#downsample to fix class imbalance


In [65]:
# clf = MLPClassifier(solver ='adam',
#                    activation = 'relu',
#                    alpha = 0.001,
#                    hidden_layer_sizes = (512,128,10),
#                    random_state = 1,
#                    max_iter = 100,
#                    verbose = True)

In [66]:
# clf.fit(x_train, y_train)

In [67]:
# y_pred = clf.predict(x_test)

In [68]:
# print(classification_report(y_test,y_pred))

In [69]:
# clf_svc = SVC(kernel = 'rbf',
#               class_weight = 'balanced',
#               random_state = 1,
#               max_iter = 20,
#               verbose = True)

In [70]:
# clf_svc.fit(x_train,y_train)

In [71]:
# y_pred_svc = clf_svc.predict(x_test)

In [72]:
# print(classification_report(y_test,y_pred_svc))

### Canny Edge Detection

In [73]:
# imgs[0]

In [119]:
canny_feats = np.zeros(imgs.shape)
for im in range(0,imgs.shape[0]):
    edges = cv2.Canny(np.uint8(imgs[im]*255),100,200)
    canny_feats[im,:,:] = edges

In [120]:
canny_feats.shape

(2966, 128, 128)

In [121]:
canny_ = canny_feats.reshape(canny_feats.shape[0], canny_feats.shape[1]*canny_feats.shape[2])
canny_.shape

(2966, 16384)

### Test HOG and Canny Features

In [122]:
feats = np.concatenate([hog_feats,canny_], axis = 1)

In [123]:
def train_model(features, labels = labels, model_type = 'mlp'):
    
    x_train,x_test,y_train,y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    if model_type == 'mlp' or 'all':
        clf = MLPClassifier(solver ='lbfgs',
                       activation = 'relu',
                       alpha = 50,
                       hidden_layer_sizes = (256,128,5),
                       random_state = 1,
                       max_iter = 500,
                       early_stopping = True,
                       verbose = True)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print('-'*30)
        print('MLP Results:')
        print(classification_report(y_test,y_pred))
    if model_type == 'all':
        print('-'*30)
        print('-'*30)
    if model_type == 'svc' or 'all':
        clf_svc = make_pipeline(StandardScaler(), SVC(kernel = 'rbf',
#                   class_weight = 'balanced',
                  random_state = 1,
                  max_iter = 100,
                  verbose = True))
        clf_svc.fit(x_train,y_train)
        y_pred_svc = clf_svc.predict(x_test)
        print('SVC Results:')
        print(classification_report(y_test,y_pred_svc))

In [124]:
#hog features
train_model(hog_feats, model_type = 'all')

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` para

------------------------------
MLP Results:
              precision    recall  f1-score   support

           1       0.31      0.42      0.35       156
          17       0.00      0.00      0.00        89
          18       0.22      0.13      0.16        87
          27       0.40      0.51      0.45       140
          64       0.37      0.46      0.41       122

    accuracy                           0.34       594
   macro avg       0.26      0.30      0.27       594
weighted avg       0.28      0.34      0.31       594

------------------------------
------------------------------
[LibSVM]

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVC Results:
              precision    recall  f1-score   support

           1       0.32      0.13      0.19       156
          17       0.31      0.16      0.21        89
          18       0.36      0.29      0.32        87
          27       0.50      0.02      0.04       140
          64       0.24      0.81      0.37       122

    accuracy                           0.27       594
   macro avg       0.35      0.28      0.23       594
weighted avg       0.35      0.27      0.21       594



In [125]:
#canny features
train_model(canny_, model_type = 'all')

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


------------------------------
MLP Results:
              precision    recall  f1-score   support

           1       0.38      0.22      0.28       156
          17       0.00      0.00      0.00        89
          18       0.26      0.37      0.30        87
          27       0.40      0.37      0.39       140
          64       0.26      0.52      0.34       122

    accuracy                           0.31       594
   macro avg       0.26      0.30      0.26       594
weighted avg       0.29      0.31      0.28       594

------------------------------
------------------------------
[LibSVM]

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVC Results:
              precision    recall  f1-score   support

           1       0.21      0.21      0.21       156
          17       0.24      0.19      0.21        89
          18       0.19      0.16      0.17        87
          27       0.30      0.20      0.24       140
          64       0.20      0.34      0.25       122

    accuracy                           0.22       594
   macro avg       0.23      0.22      0.22       594
weighted avg       0.23      0.22      0.22       594



In [126]:
#both hog + canny
train_model(feats, model_type = 'all')

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


------------------------------
MLP Results:
              precision    recall  f1-score   support

           1       0.28      0.28      0.28       156
          17       0.24      0.20      0.22        89
          18       0.22      0.11      0.15        87
          27       0.37      0.30      0.33       140
          64       0.24      0.40      0.30       122

    accuracy                           0.27       594
   macro avg       0.27      0.26      0.26       594
weighted avg       0.28      0.27      0.27       594

------------------------------
------------------------------
[LibSVM]

/Users/Courtney/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVC Results:
              precision    recall  f1-score   support

           1       0.34      0.32      0.33       156
          17       0.31      0.30      0.31        89
          18       0.31      0.22      0.26        87
          27       0.13      0.01      0.03       140
          64       0.20      0.47      0.28       122

    accuracy                           0.26       594
   macro avg       0.26      0.26      0.24       594
weighted avg       0.26      0.26      0.23       594

